In [1]:
# from ortools.sat.python import cp_model
from ortools.linear_solver.pywraplp import Solver

Make a variable for each nurse patient pair
Keep track of the total number of patients each nurse sees
Maximize the acuity of each nurse for each patient type
Max and min number of patients per nurse
Need to create a dictionary for patients so that each type maps to a number of patients

In [2]:
from typing import List

class NursePatientMatcher:
    
    def __init__(self, num_patients, num_nurses, max_patients, min_patients, patient_types, acuities):
        self.p: int = num_patients
        self.n: int = num_nurses
        self.max_patients_per_nurse: int = max_patients
        self.min_patients_per_nurse: int = min_patients
        self.patients_of_each_type: List[int] = patient_types
        self.patient_nurse_acuities: List[List[int]] = acuities

In [3]:
def create_variables(self):
    model: Solver = self.model
    n: int = self.n
    p: int = self.p
    patient_types = self.patients_of_each_type
    max_patients: int = self.max_patients_per_nurse
    self.x = {}
    self.patients_per_nurse = {}
    self.patient_types = {}
    for nurse in range(n):
        for patient in range(p):
            self.x[nurse, patient] = model.IntVar(0, 1, f'x[Nurse: {nurse}, Patient: {patient}]')
    
    for nurse in range(n):
        self.patients_per_nurse[nurse] = model.IntVar(0, max_patients, f'Patients for Nurse {nurse}')
    
    prev = 0
    for t in range(len(patient_types)):
        num_patients_of_type = patient_types[t]
        self.patient_types[t] = set(range(prev, prev + num_patients_of_type))
        prev = prev + num_patients_of_type

# Register this method with the solver class
NursePatientMatcher.create_variables = create_variables

In [4]:
def bound_patients_per_nurse(self):
    x = self.x
    model: Solver = self.model
    n: int = self.n
    p: int = self.p
    max_patients: int = self.max_patients_per_nurse
    min_patients: int = self.min_patients_per_nurse
    for nurse in range(n):
        model.Add(sum(x[nurse,patient] for patient in range(p)) <= max_patients)
        model.Add(sum(x[nurse,patient] for patient in range(p)) >= min_patients)

# Register this method with the solver class
NursePatientMatcher.bound_patients_per_nurse = bound_patients_per_nurse

In [5]:
def one_nurse_per_patient(self):
    x = self.x
    model: Solver = self.model
    n: int = self.n
    p: int = self.p
    for patient in range(p):
        model.Add(sum(x[nurse,patient] for nurse in range(n)) == 1)


# Register this method with the solver class
NursePatientMatcher.one_nurse_per_patient = one_nurse_per_patient

In [6]:
def track_patients_per_nurse(self):
    x = self.x
    model: Solver = self.model
    n: int = self.n
    p: int = self.p
    patients_per_nurse = self.patients_per_nurse
    for nurse in range(n):
        model.Add(sum(x[nurse,patient] for patient in range(p)) == patients_per_nurse[nurse])


# Register this method with the solver class
NursePatientMatcher.track_patients_per_nurse = track_patients_per_nurse

In [7]:
def minimize_objectives(self):
    x = self.x
    model: Solver = self.model
    patients_per_nurse = self.patients_per_nurse
    n: int = self.n
    p: int = self.p
    max_patients: int = self.max_patients_per_nurse
    min_patients: int = self.min_patients_per_nurse

    avg_workload = model.IntVar(min_patients, max_patients, 'Average Workload of Nurses')

    model.Add(sum(patients_per_nurse[nurse] for nurse in range(n))/n == avg_workload)

    # absolute_diff = [model.IntVar(0, max_patients, "Absolute difference between nurse n workload and average workload") for _ in range(n)]

    helpers = [model.IntVar(0, max_patients, 'Helper variable to keep things linear') for _ in range(n)]

    for nurse in range(n):
        # difference = model.IntVar(-max_patients, max_patients, 'Difference between nurse n workload and average workload')
        # model.Add(difference == patients_per_nurse[nurse] - avg_workload)
        model.Add(helpers[nurse] >= patients_per_nurse[nurse] - avg_workload)
        model.Add(helpers[nurse] <= -(patients_per_nurse[nurse] - avg_workload))
    
    model.Minimize(sum(helpers))
    

# Register this method with the solver class
NursePatientMatcher.minimize_objectives = minimize_objectives

In [8]:
def solve(self):
    self.model = Solver('NursePatientMatcher', Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    self.create_variables()
    self.bound_patients_per_nurse()
    self.one_nurse_per_patient()
    self.track_patients_per_nurse()
        
    self.minimize_objectives()
    if self.model.Solve() == Solver.OPTIMAL:
        print('optimal')
    else:   
        raise ValueError('Modeling error!')

# Register this method with the solver class
NursePatientMatcher.solve = solve

In [9]:
with open('data/3nurse5patientType0.txt') as f:
    lines = f.readlines()
    num_nurses, num_patients, num_patient_types = lines[0].split(' ')
    num_nurses, num_patients, num_patient_types = int(num_nurses), int(num_patients), int(num_patient_types)

    min_patients, max_patients = lines[1].split(' ')
    min_patients, max_patients = int(min_patients), int(max_patients)

    patient_types = lines[3].strip().split(' ')
    patient_types = [int(x) for x in patient_types]

    acuities = []
    for i in range(num_nurses):
        nurse_acuities = lines[5 + i].strip().split(' ')
        acuities.append([int(x) for x in nurse_acuities])

matcher1 = NursePatientMatcher(num_patients, num_nurses, max_patients, min_patients, patient_types, acuities)
soln1 = matcher1.solve()

Error: Canceled future for execute_request message before replies were done